#Preparation

In [12]:
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 torchtext==0.14.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 101.3 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.2
    Uninstalling torchvision-0.15.2:
      Successfully uninstalled torchvision-0.15.2
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.2
    Uninstalling torchtext-0.15.2:
      Successfully uninstalled torchtext-0.15.2
  Attempting uninstall: torchdata
    Found existing installation: torchdata 0.6.1
    Uninstalling torchda

In [1]:
!git clone https://github.com/pliang279/MultiBench

Cloning into 'MultiBench'...
remote: Enumerating objects: 6937, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 6937 (delta 68), reused 121 (delta 60), pack-reused 6789
Receiving objects: 100% (6937/6937), 51.07 MiB | 21.70 MiB/s, done.
Resolving deltas: 100% (4254/4254), done.


In [1]:
%cd MultiBench/

/content/MultiBench


In [3]:
!pip install gdown

!gdown https://drive.google.com/uc?id=1_XdzdW8UNG1TTS6QcX10uhoS6N11OBit&export=download #MOSI data
!gdown https://drive.google.com/uc?id=180l4pN6XAv8-OAYQ6OrMheFUMwtqUWbz&export=download #MOSEI data
!gdown https://drive.google.com/uc?id=1EMBUmUL5B0PTncGx3L-sBElGOmjFBR_h&export=download #Sarcasm data
!gdown https://drive.google.com/uc?id=1L5slPmYyhEVtwGyM1kgcFMjeBpXLZGT0&export=download #Humor data
!gdown https://drive.google.com/uc?id=1SuTPg0MTo4P8dXLKFjK7LSoXWqm1YP92&export=download #MIMIC data


Downloading...
From: https://drive.google.com/uc?id=1_XdzdW8UNG1TTS6QcX10uhoS6N11OBit
To: /content/MultiBench/mosi_data.pkl
100% 154M/154M [00:03<00:00, 42.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=180l4pN6XAv8-OAYQ6OrMheFUMwtqUWbz
To: /content/MultiBench/mosei_senti_data.pkl
100% 3.73G/3.73G [00:44<00:00, 84.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EMBUmUL5B0PTncGx3L-sBElGOmjFBR_h
To: /content/MultiBench/sarcasm.pkl
100% 208M/208M [00:00<00:00, 232MB/s]
Downloading...
From: https://drive.google.com/uc?id=1L5slPmYyhEVtwGyM1kgcFMjeBpXLZGT0
To: /content/MultiBench/humor.pkl
100% 1.22G/1.22G [00:09<00:00, 122MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SuTPg0MTo4P8dXLKFjK7LSoXWqm1YP92
To: /content/MultiBench/im.pk
100% 83.2M/83.2M [00:00<00:00, 190MB/s]


In [4]:
!git clone https://github.com/pliang279/FactorCL

Cloning into 'FactorCL'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 128 (delta 2), reused 0 (delta 0), pack-reused 116
Receiving objects: 100% (128/128), 288.20 KiB | 1.43 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
import os
import sys
from torch.utils.data import DataLoader
from sklearn.linear_model import LogisticRegression

from unimodals.common_models import Transformer, MLP
from unimodals.common_models import MLP, GRUWithLinear, GRU


In [3]:
sys.path.append(os.getcwd())

In [4]:
%cd FactorCL

/content/MultiBench/FactorCL


In [5]:
from multibench_model import*

#MOSI

In [57]:
from datasets.affect.get_data import get_dataloader

train_loader, valid_loader, test_loader = get_dataloader('/content/MultiBench/mosi_data.pkl',
                                                         robust_test=False,
                                                         batch_size=32,
                                                         train_shuffle=True)

eval_train_loader, eval_valid_loader, eval_test_loader = get_dataloader('/content/MultiBench/mosi_data.pkl',
                                                                        robust_test=False,
                                                                        batch_size=32,
                                                                        train_shuffle=False)

##FactorCL-SUP

In [58]:
encoders = [Transformer(20, 40), Transformer(300, 600)]
factorcl_sup = FactorCLSUP(encoders=encoders, feat_dims=[40, 600], y_ohe_dim=3).cuda()
train_sup_mosi(factorcl_sup, train_loader, num_epoch=100, num_club_iter=1)

iter:  0  i_batch:  0  loss:  0.0011624246835708618
iter:  1  i_batch:  0  loss:  -0.03190102055668831
iter:  2  i_batch:  0  loss:  -0.4311522841453552
iter:  3  i_batch:  0  loss:  -0.9014155864715576
iter:  4  i_batch:  0  loss:  -2.1317553520202637
iter:  5  i_batch:  0  loss:  -2.976452350616455
iter:  6  i_batch:  0  loss:  -3.3434486389160156
iter:  7  i_batch:  0  loss:  -3.346254348754883
iter:  8  i_batch:  0  loss:  -4.042529106140137
iter:  9  i_batch:  0  loss:  -4.252300262451172
iter:  10  i_batch:  0  loss:  -4.225554466247559
iter:  11  i_batch:  0  loss:  1.0005271434783936
iter:  12  i_batch:  0  loss:  -4.045951843261719
iter:  13  i_batch:  0  loss:  -4.469624996185303
iter:  14  i_batch:  0  loss:  -4.786789894104004
iter:  15  i_batch:  0  loss:  -4.101232528686523
iter:  16  i_batch:  0  loss:  -4.211440086364746
iter:  17  i_batch:  0  loss:  -3.848883628845215
iter:  18  i_batch:  0  loss:  -3.969515085220337
iter:  19  i_batch:  0  loss:  -4.6729512214660645


In [ ]:
factorcl_sup.eval()

train_embeds_x1 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_train_loader])
train_labels = mosi_label(train_labels)

test_embeds_x1 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_test_loader])
test_embeds_x2 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_test_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_test_loader])
test_labels = mosi_label(test_labels)

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
score

0.6720116618075802

##FactorCL-SSL

In [ ]:
encoders = [Transformer(20, 40), Transformer(300, 600)]
factorcl_ssl = FactorCLSSL(encoders=encoders, feat_dims=[40, 600], y_ohe_dim=3).cuda()
train_ssl_mosi(factorcl_ssl, train_loader, num_epoch=100, num_club_iter=1)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


iter:  0  i_batch:  0  loss:  -0.002487795427441597
iter:  1  i_batch:  0  loss:  -1.830185890197754
iter:  2  i_batch:  0  loss:  -3.1662981510162354
iter:  3  i_batch:  0  loss:  -3.681149482727051
iter:  4  i_batch:  0  loss:  -4.313801288604736
iter:  5  i_batch:  0  loss:  -4.4881367683410645
iter:  6  i_batch:  0  loss:  -4.696531295776367
iter:  7  i_batch:  0  loss:  -4.604437351226807
iter:  8  i_batch:  0  loss:  -6.114076614379883
iter:  9  i_batch:  0  loss:  -6.7265400886535645
iter:  10  i_batch:  0  loss:  -6.742205619812012
iter:  11  i_batch:  0  loss:  -7.247134208679199
iter:  12  i_batch:  0  loss:  -8.011838912963867
iter:  13  i_batch:  0  loss:  -8.293033599853516
iter:  14  i_batch:  0  loss:  -5.3193159103393555
iter:  15  i_batch:  0  loss:  -8.390777587890625
iter:  16  i_batch:  0  loss:  -8.524253845214844
iter:  17  i_batch:  0  loss:  -7.289597511291504
iter:  18  i_batch:  0  loss:  -9.07352066040039
iter:  19  i_batch:  0  loss:  -7.363587379455566
iter

In [ ]:
factorcl_ssl.eval()

train_embeds_x1 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_train_loader])
train_labels = mosi_label(train_labels)

test_embeds_x1 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_test_loader])
test_embeds_x2 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_test_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_test_loader])
test_labels = mosi_label(test_labels)

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
score

0.5408163265306123

##SupCon

In [61]:
encoders = [Transformer(20, 40), Transformer(300, 40)]

supcon_model = SupConModel(temperature=0.5, encoders=encoders, dim_ins=[40, 40], feat_dims=[40, 40], use_label=True).cuda()
supcon_optim = optim.Adam(supcon_model.parameters())
train_supcon_mosi(supcon_model, train_loader, supcon_optim, num_epoch=10)

iter:  0  i_batch:  0  loss:  40.36393737792969
iter:  1  i_batch:  0  loss:  29.615131378173828
iter:  2  i_batch:  0  loss:  29.602519989013672
iter:  3  i_batch:  0  loss:  29.601627349853516
iter:  4  i_batch:  0  loss:  29.598114013671875
iter:  5  i_batch:  0  loss:  29.59786033630371
iter:  6  i_batch:  0  loss:  29.591426849365234
iter:  7  i_batch:  0  loss:  29.59424591064453
iter:  8  i_batch:  0  loss:  29.598228454589844
iter:  9  i_batch:  0  loss:  29.596210479736328


In [ ]:
supcon_model.eval()

train_embeds_x1 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_train_loader])
train_labels = mosi_label(train_labels)

test_embeds_x1 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_test_loader])
test_embeds_x2 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_test_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_test_loader])
test_labels = mosi_label(test_labels)

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
score

0.4620991253644315

#Humor

In [54]:
from datasets.affect.get_data import get_dataloader

train_loader, valid_loader, test_loader = get_dataloader('/content/MultiBench/humor.pkl',
                                                         batch_size=128,
                                                         data_type='humor')

eval_train_loader, eval_valid_loader, eval_test_loader = get_dataloader('/content/MultiBench/humor.pkl',
                                                                        batch_size=128,
                                                                        data_type='humor',
                                                                        train_shuffle=False)

##FactorCL-SUP

In [55]:
encoders = [Transformer(371, 40), Transformer(300, 40)]
factorcl_sup = FactorCLSUP(encoders=encoders, feat_dims=[40, 40], y_ohe_dim=3).cuda()
train_sup_sarcasm(factorcl_sup, train_loader, num_epoch=100, num_club_iter=1)

iter:  0  i_batch:  0  loss:  0.0013007273664698005
iter:  1  i_batch:  0  loss:  -0.1793300211429596
iter:  2  i_batch:  0  loss:  -0.36496207118034363
iter:  3  i_batch:  0  loss:  -0.05508889630436897
iter:  4  i_batch:  0  loss:  0.24322691559791565
iter:  5  i_batch:  0  loss:  0.6587186455726624
iter:  6  i_batch:  0  loss:  1.0358939170837402
iter:  7  i_batch:  0  loss:  1.116131067276001
iter:  8  i_batch:  0  loss:  1.1662193536758423
iter:  9  i_batch:  0  loss:  1.4159873723983765
iter:  10  i_batch:  0  loss:  1.4103529453277588
iter:  11  i_batch:  0  loss:  1.6665235757827759
iter:  12  i_batch:  0  loss:  1.6012569665908813
iter:  13  i_batch:  0  loss:  1.6238175630569458
iter:  14  i_batch:  0  loss:  1.7627021074295044
iter:  15  i_batch:  0  loss:  1.966027855873108
iter:  16  i_batch:  0  loss:  1.733707070350647
iter:  17  i_batch:  0  loss:  1.935907244682312
iter:  18  i_batch:  0  loss:  2.0887207984924316
iter:  19  i_batch:  0  loss:  2.180851697921753
iter: 

In [ ]:
factorcl_sup.eval()

train_embeds_x1 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_train_loader])
train_labels = sarcasm_label(train_labels)

test_embeds_x1 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_test_loader])
test_embeds_x2 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_test_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_test_loader])
test_labels = sarcasm_label(test_labels)

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

In [ ]:
score

0.6294896030245747

##FactorCL-SSL

In [ ]:
encoders = [Transformer(371, 40), Transformer(300, 40)]
factorcl_ssl = FactorCLSSL(encoders=encoders, feat_dims=[40, 40], y_ohe_dim=3).cuda()
train_ssl_sarcasm(factorcl_ssl, train_loader, num_epoch=100, num_club_iter=1)

In [ ]:
factorcl_ssl.eval()

train_embeds_x1 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_train_loader])
train_labels = sarcasm_label(train_labels)

test_embeds_x1 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_test_loader])
test_embeds_x2 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_test_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_test_loader])
test_labels = sarcasm_label(test_labels)

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

In [ ]:
score

0.5964083175803403

##SupCon

In [ ]:
encoders = [Transformer(371, 40), Transformer(300, 40)]

# set use_label=False for SimCLR
supcon_model = SupConModel(temperature=0.5, encoders=encoders, dim_ins=[40, 40], feat_dims=[40, 40], use_label=True).cuda()

supcon_optim = optim.Adam(supcon_model.parameters())
train_supcon_sarcasm(supcon_model, train_loader, supcon_optim, modalities=[0,2], num_epoch=100)

In [ ]:
supcon_model.eval()

train_embeds_x1 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_train_loader])
train_labels = sarcasm_label(train_labels)

test_embeds_x1 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_test_loader])
test_embeds_x2 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_test_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_test_loader])
test_labels = sarcasm_label(test_labels)

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

In [ ]:
score

0.5160680529300568

#MIMIC

In [64]:
from datasets.mimic.get_data import get_dataloader

train_loader, valid_loader, test_loader = get_dataloader(
    7, imputed_path='/content/MultiBench/im.pk')

eval_train_loader, eval_valid_loader, eval_test_loader = get_dataloader(
    7, imputed_path='/content/MultiBench/im.pk', train_shuffle=False)

100%|██████████| 11/11 [00:05<00:00,  2.05it/s]


##FactorCL-SUP

In [65]:
#encoders = [MLP(5, 10, 10, dropout=False),
#            GRUWithLinear(12, 30, 15, flatten=True, batch_first=True)]

encoders = [MLP(5, 10, 10, dropout=False),
            GRU(12, 30, dropout=False, batch_first=True, flatten=True)]

factorcl_sup = FactorCLSUP(encoders=encoders, feat_dims=[10, 720], y_ohe_dim=2).cuda()
train_sup_mimic(factorcl_sup, train_loader, num_epoch=30, num_club_iter=1)

iter:  0  i_batch:  0  loss:  -0.0012989966198801994
iter:  0  i_batch:  100  loss:  -0.2463875561952591
iter:  0  i_batch:  200  loss:  -0.5832105875015259
iter:  0  i_batch:  300  loss:  -0.6508282423019409
iter:  0  i_batch:  400  loss:  -0.7278008460998535
iter:  0  i_batch:  500  loss:  -0.7705122232437134
iter:  0  i_batch:  600  loss:  -0.8182592391967773
iter:  1  i_batch:  0  loss:  -1.0183138847351074
iter:  1  i_batch:  100  loss:  -0.8504148125648499
iter:  1  i_batch:  200  loss:  -0.8186317682266235
iter:  1  i_batch:  300  loss:  -1.3324253559112549
iter:  1  i_batch:  400  loss:  -1.222513198852539
iter:  1  i_batch:  500  loss:  -0.9067964553833008
iter:  1  i_batch:  600  loss:  -0.8614742159843445
iter:  2  i_batch:  0  loss:  -1.2362432479858398
iter:  2  i_batch:  100  loss:  -0.9707288146018982
iter:  2  i_batch:  200  loss:  -0.9806453585624695
iter:  2  i_batch:  300  loss:  -1.0745757818222046
iter:  2  i_batch:  400  loss:  -0.6654452681541443
iter:  2  i_batc

In [ ]:
factorcl_sup.eval()

train_embeds_x1 = np.concatenate([factorcl_sup.get_embedding(data[0].float().cuda(), data[1].float().cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([factorcl_sup.get_embedding(data[0].float().cuda(), data[1].float().cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[2].detach().cpu().numpy() for data in eval_train_loader])

test_embeds_x1 = np.concatenate([factorcl_sup.get_embedding(data[0].float().cuda(), data[1].float().cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
test_embeds_x2 = np.concatenate([factorcl_sup.get_embedding(data[0].float().cuda(), data[1].float().cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[2].detach().cpu().numpy() for data in eval_train_loader])

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
score

0.7897903652320546

##FactorCL-SSL

In [ ]:
#encoders = [MLP(5, 10, 10, dropout=False),
#            GRUWithLinear(12, 30, 15, flatten=True, batch_first=True)]

encoders = [MLP(5, 10, 10, dropout=False),
            GRU(12, 30, dropout=False, batch_first=True, flatten=True)]

factorcl_ssl = FactorCLSSL(encoders=encoders, feat_dims=[10, 720], y_ohe_dim=2).cuda()
train_ssl_mimic(factorcl_ssl, train_loader, num_epoch=30, num_club_iter=1)

iter:  0  i_batch:  0  loss:  0.0006223861128091812
iter:  0  i_batch:  100  loss:  -3.5920844078063965
iter:  0  i_batch:  200  loss:  -4.292657852172852
iter:  0  i_batch:  300  loss:  -4.513991355895996
iter:  0  i_batch:  400  loss:  -5.4547119140625
iter:  0  i_batch:  500  loss:  -5.684263706207275
iter:  0  i_batch:  600  loss:  -3.789016008377075
iter:  1  i_batch:  0  loss:  -5.70589542388916
iter:  1  i_batch:  100  loss:  -6.036656856536865
iter:  1  i_batch:  200  loss:  -5.840413570404053
iter:  1  i_batch:  300  loss:  -6.080589294433594
iter:  1  i_batch:  400  loss:  -5.495912551879883
iter:  1  i_batch:  500  loss:  -5.943778038024902
iter:  1  i_batch:  600  loss:  -7.005795955657959
iter:  2  i_batch:  0  loss:  -5.923552989959717
iter:  2  i_batch:  100  loss:  -6.397305488586426
iter:  2  i_batch:  200  loss:  -7.055599212646484
iter:  2  i_batch:  300  loss:  -6.3207011222839355
iter:  2  i_batch:  400  loss:  -6.897139072418213
iter:  2  i_batch:  500  loss:  -6.

In [ ]:
factorcl_ssl.eval()

train_embeds_x1 = np.concatenate([factorcl_ssl.get_embedding(data[0].float().cuda(), data[1].float().cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([factorcl_ssl.get_embedding(data[0].float().cuda(), data[1].float().cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[2].detach().cpu().numpy() for data in eval_train_loader])

test_embeds_x1 = np.concatenate([factorcl_ssl.get_embedding(data[0].float().cuda(), data[1].float().cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
test_embeds_x2 = np.concatenate([factorcl_ssl.get_embedding(data[0].float().cuda(), data[1].float().cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[2].detach().cpu().numpy() for data in eval_train_loader])

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
score

0.6653891848388457

##SupCon

In [ ]:
#encoders = [MLP(5, 10, 10, dropout=False),
#            GRUWithLinear(12, 30, 15, flatten=True, batch_first=True)]

encoders = [MLP(5, 10, 10, dropout=False),
            GRU(12, 30, dropout=False, batch_first=True, flatten=True)]

supcon_model = SupConModel(temperature=0.5, encoders=encoders, dim_ins=[10, 720], feat_dims=[40, 40], use_label=True).cuda()
supcon_optim = optim.Adam(supcon_model.parameters())
train_supcon_mimic(supcon_model, train_loader, supcon_optim, num_epoch=5)

iter:  0  i_batch:  0  loss:  33.60186767578125
iter:  0  i_batch:  100  loss:  31.204633712768555
iter:  0  i_batch:  200  loss:  31.2003116607666
iter:  0  i_batch:  300  loss:  31.190750122070312
iter:  0  i_batch:  400  loss:  31.21358299255371
iter:  0  i_batch:  500  loss:  31.224945068359375
iter:  0  i_batch:  600  loss:  31.217573165893555
iter:  1  i_batch:  0  loss:  31.2016544342041
iter:  1  i_batch:  100  loss:  31.154462814331055
iter:  1  i_batch:  200  loss:  31.185989379882812
iter:  1  i_batch:  300  loss:  31.21261978149414
iter:  1  i_batch:  400  loss:  31.23200035095215
iter:  1  i_batch:  500  loss:  31.17060661315918
iter:  1  i_batch:  600  loss:  31.194486618041992
iter:  2  i_batch:  0  loss:  31.2025089263916
iter:  2  i_batch:  100  loss:  31.1802978515625
iter:  2  i_batch:  200  loss:  31.162372589111328
iter:  2  i_batch:  300  loss:  31.20770263671875
iter:  2  i_batch:  400  loss:  31.207172393798828
iter:  2  i_batch:  500  loss:  31.17645835876465
i

In [ ]:
supcon_model.eval()

train_embeds_x1 = np.concatenate([supcon_model.get_embedding(data[0].float().cuda(), data[1].float().cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([supcon_model.get_embedding(data[0].float().cuda(), data[1].float().cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[2].detach().cpu().numpy() for data in eval_train_loader])

test_embeds_x1 = np.concatenate([supcon_model.get_embedding(data[0].float().cuda(), data[1].float().cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
test_embeds_x2 = np.concatenate([supcon_model.get_embedding(data[0].float().cuda(), data[1].float().cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[2].detach().cpu().numpy() for data in eval_train_loader])

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
score

0.6641628022841375

#Sarcasm

In [6]:
from datasets.affect.get_data import get_dataloader

train_loader, valid_loader, test_loader = get_dataloader('/content/MultiBench/sarcasm.pkl',
                                                         batch_size=128,
                                                         data_type='sarcasm')

eval_train_loader, eval_valid_loader, eval_test_loader = get_dataloader('/content/MultiBench/sarcasm.pkl',
                                                                        batch_size=128,
                                                                        data_type='sarcasm',
                                                                        train_shuffle=False)

##FactorCL-SUP

In [40]:
encoders = [Transformer(371, 40), Transformer(300, 40)]
factorcl_sup = FactorCLSUP(encoders=encoders, feat_dims=[40, 40], y_ohe_dim=3).cuda()
train_sup_sarcasm(factorcl_sup, train_loader, num_epoch=100, num_club_iter=1)

iter:  0  i_batch:  0  loss:  0.00022673769854009151
iter:  1  i_batch:  0  loss:  -0.010921637527644634
iter:  2  i_batch:  0  loss:  -0.019974887371063232
iter:  3  i_batch:  0  loss:  -0.028301645070314407
iter:  4  i_batch:  0  loss:  -0.03775062412023544
iter:  5  i_batch:  0  loss:  -0.05263204872608185
iter:  6  i_batch:  0  loss:  -0.06939707696437836
iter:  7  i_batch:  0  loss:  -0.09028887748718262
iter:  8  i_batch:  0  loss:  -0.12177552282810211
iter:  9  i_batch:  0  loss:  -0.1584542989730835
iter:  10  i_batch:  0  loss:  -0.20651768147945404
iter:  11  i_batch:  0  loss:  -0.24353653192520142
iter:  12  i_batch:  0  loss:  -0.29933953285217285
iter:  13  i_batch:  0  loss:  -0.39696305990219116
iter:  14  i_batch:  0  loss:  -0.4287801682949066
iter:  15  i_batch:  0  loss:  -0.4027155935764313
iter:  16  i_batch:  0  loss:  -0.44266486167907715
iter:  17  i_batch:  0  loss:  -0.5260065197944641
iter:  18  i_batch:  0  loss:  -0.5915610194206238
iter:  19  i_batch:  0

In [52]:
factorcl_sup.eval()

train_embeds_x1 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_train_loader])
train_labels = sarcasm_label(train_labels)

test_embeds_x1 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_test_loader])
test_embeds_x2 = np.concatenate([factorcl_sup.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_test_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_test_loader])
test_labels = sarcasm_label(test_labels)

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [53]:
score

0.6666666666666666

##FactorCL-SSL

In [ ]:
encoders = [Transformer(371, 40), Transformer(300, 40)]
factorcl_ssl = FactorCLSSL(encoders=encoders, feat_dims=[40, 40], y_ohe_dim=3).cuda()
train_ssl_sarcasm(factorcl_ssl, train_loader, num_epoch=100, num_club_iter=1)

iter:  0  i_batch:  0  loss:  0.0006452705711126328
iter:  1  i_batch:  0  loss:  -0.012466572225093842
iter:  2  i_batch:  0  loss:  -0.042170800268650055
iter:  3  i_batch:  0  loss:  -0.08974642306566238
iter:  4  i_batch:  0  loss:  -0.18145647644996643
iter:  5  i_batch:  0  loss:  -0.3081078827381134
iter:  6  i_batch:  0  loss:  -0.4158934950828552
iter:  7  i_batch:  0  loss:  -0.601329505443573
iter:  8  i_batch:  0  loss:  -0.7000826597213745
iter:  9  i_batch:  0  loss:  -0.8313818573951721
iter:  10  i_batch:  0  loss:  -0.9973404407501221
iter:  11  i_batch:  0  loss:  -1.053339958190918
iter:  12  i_batch:  0  loss:  -1.1727815866470337
iter:  13  i_batch:  0  loss:  -1.2500641345977783
iter:  14  i_batch:  0  loss:  -1.4549574851989746
iter:  15  i_batch:  0  loss:  -1.601521372795105
iter:  16  i_batch:  0  loss:  -1.6972362995147705
iter:  17  i_batch:  0  loss:  -1.7666714191436768
iter:  18  i_batch:  0  loss:  -2.0024752616882324
iter:  19  i_batch:  0  loss:  -2.24

In [ ]:
factorcl_ssl.eval()

train_embeds_x1 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_train_loader])
train_labels = sarcasm_label(train_labels)

test_embeds_x1 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_test_loader])
test_embeds_x2 = np.concatenate([factorcl_ssl.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_test_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_test_loader])
test_labels = sarcasm_label(test_labels)

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
score

0.6014492753623188

##SupCon

In [ ]:
encoders = [Transformer(371, 40), Transformer(300, 40)]

# set use_label=False for SimCLR
supcon_model = SupConModel(temperature=0.5, encoders=encoders, dim_ins=[40, 40], feat_dims=[40, 40], use_label=True).cuda()

supcon_optim = optim.Adam(supcon_model.parameters())
train_supcon_sarcasm(supcon_model, train_loader, supcon_optim, modalities=[0,2], num_epoch=20)

iter:  0  i_batch:  0  loss:  49.70327377319336
iter:  1  i_batch:  0  loss:  39.85050964355469
iter:  2  i_batch:  0  loss:  39.70650863647461
iter:  3  i_batch:  0  loss:  39.652442932128906
iter:  4  i_batch:  0  loss:  39.626487731933594
iter:  5  i_batch:  0  loss:  39.60759353637695
iter:  6  i_batch:  0  loss:  39.59840393066406
iter:  7  i_batch:  0  loss:  39.597259521484375
iter:  8  i_batch:  0  loss:  39.5950927734375
iter:  9  i_batch:  0  loss:  39.590240478515625
iter:  10  i_batch:  0  loss:  39.590660095214844
iter:  11  i_batch:  0  loss:  39.59142303466797
iter:  12  i_batch:  0  loss:  39.58800506591797
iter:  13  i_batch:  0  loss:  39.587642669677734
iter:  14  i_batch:  0  loss:  39.589324951171875
iter:  15  i_batch:  0  loss:  39.58818054199219
iter:  16  i_batch:  0  loss:  39.58576202392578
iter:  17  i_batch:  0  loss:  39.58720397949219
iter:  18  i_batch:  0  loss:  39.5872802734375
iter:  19  i_batch:  0  loss:  39.585548400878906


In [ ]:
supcon_model.eval()

train_embeds_x1 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_train_loader])
train_embeds_x2 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_train_loader])
train_embeds = np.concatenate([train_embeds_x1, train_embeds_x2], axis=1)
train_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_train_loader])
train_labels = sarcasm_label(train_labels)

test_embeds_x1 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[0].detach().cpu().numpy() for data in eval_test_loader])
test_embeds_x2 = np.concatenate([supcon_model.get_embedding(data[0][0].cuda(), data[0][2].cuda())[1].detach().cpu().numpy() for data in eval_test_loader])
test_embeds = np.concatenate([test_embeds_x1, test_embeds_x2], axis=1)
test_labels = np.concatenate([data[3].detach().cpu().numpy() for data in eval_test_loader])
test_labels = sarcasm_label(test_labels)

# Train Logistic Classifier
clf = LogisticRegression(max_iter=200).fit(train_embeds, train_labels)
score = clf.score(test_embeds, test_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
score

0.572463768115942